In [7]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install selenium

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time

Path to your ChromeDriver
CHROME_DRIVER_PATH = r"C:\Users\HP\Downloads\SHL\chromedriver-win64\chromedriver.exe"




In [18]:
CHROME_DRIVER_PATH

'C:\\Users\\HP\\Downloads\\SHL\\chromedriver-win64\\chromedriver.exe'

In [19]:
service = Service(CHROME_DRIVER_PATH)
driver = webdriver.Chrome(service=service)

In [12]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

# Initialize Edge WebDriver
driver = webdriver.Edge()

# Open the target page
url = 'https://www.shl.com/solutions/products/product-catalog/view/accounts-payable-new/'
driver.get(url)
driver.maximize_window()

wait = WebDriverWait(driver, 10)

# Accept cookies (if any)
try:
    cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Allow all cookies')]")))
    cookie_btn.click()
except:
    pass

# Title
try:
    title = driver.find_element(By.TAG_NAME, "h1").text.strip()
except:
    title = "Not found"

# Description
try:
    desc_elem = driver.find_element(By.CSS_SELECTOR, ".product-catalogue-training-calendar__row.typ p")
    description = desc_elem.text.strip()
except:
    description = "Not found"

# Test Type
try:
    test_type_spans = driver.find_elements(By.CSS_SELECTOR, "span.product-catalogue__key")
    test_type_values = [span.text.strip() for span in test_type_spans if span.text.strip()]
    test_type = ", ".join(test_type_values) if test_type_values else "Not found"
except:
    test_type = "Not found"

# Remote Testing
try:
    remote_span = driver.find_element(By.CSS_SELECTOR, "span.catalogue__circle.-yes")
    remote_testing = "Yes"
except:
    remote_testing = "No"

# Duration (Assessment length)
try:
    duration = "Not found"
    headers = driver.find_elements(By.TAG_NAME, "h4")
    for header in headers:
        if "Assessment length" in header.text:
            sibling_p = header.find_element(By.XPATH, "following-sibling::p[1]")
            match = re.search(r'(\d+)', sibling_p.text)
            if match:
                duration = match.group(1) + " minutes"
            break
except:
    duration = "Not found"

# Final Output
print("Title:", title)
print("Description:", description)
print("Test Type:", test_type)
print("Remote Testing:", remote_testing)
print("Duration:", duration)

# Quit the browser
driver.quit()


Title: Accounts Payable (New)
Description: Multiple-choice test that measures the knowledge of processing payables and vendor invoices, and the posting of journal entries.
Test Type: K
Remote Testing: Yes
Duration: 9 minutes


In [31]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
import re

# URL to scrape (change for each product)
url = 'https://www.shl.com/solutions/products/product-catalog/view/apache-hadoop-new/'

# Initialize Edge driver
driver = webdriver.Edge()
driver.get(url)
driver.maximize_window()

wait = WebDriverWait(driver, 10)

# Accept cookies
try:
    cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Allow all cookies')]")))
    cookie_btn.click()
except:
    pass

# Extract Title
try:
    title = driver.find_element(By.TAG_NAME, "h1").text.strip()
except:
    title = "Not found"

# Extract Description
try:
    desc_elem = driver.find_element(By.CSS_SELECTOR, ".product-catalogue-training-calendar__row.typ p")
    description = desc_elem.text.strip()
except:
    description = "Not found"

# Extract Test Type
try:
    test_type_spans = driver.find_elements(By.CSS_SELECTOR, "span.product-catalogue__key")
    test_type_values = [span.text.strip() for span in test_type_spans if span.text.strip()]
    test_type = ", ".join(test_type_values)
except:
    test_type = "Not found"

# Remote Testing
try:
    driver.find_element(By.CSS_SELECTOR, "span.catalogue__circle.-yes")
    remote_testing = "Yes"
except:
    remote_testing = "No"

# Duration
try:
    assessment_blocks = driver.find_elements(By.CSS_SELECTOR, ".product-catalogue-training-calendar__row")
    duration = "Not found"
    for block in assessment_blocks:
        if "Assessment length" in block.text:
            match = re.search(r'(\d+)\s*minutes', block.text)
            if match:
                duration = match.group(1) + " minutes"
            break
except:
    duration = "Not found"

driver.quit()

# Prepare data
data = {
    'Assessment Name': title,
    'Description': description,
    'URL': url,
    'Remote Testing': remote_testing,
    'Duration': duration,
    'Test Type': test_type
}

df_new = pd.DataFrame([data])

# File path
file_path = "SHL_Assessments.xlsx"

# Append or create
if os.path.exists(file_path):
    df_existing = pd.read_excel(file_path)
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
else:
    df_combined = df_new

# Save to Excel
df_combined.to_excel(file_path, index=False)

print("✅ Data saved/appended to:", file_path)


✅ Data saved/appended to: SHL_Assessments.xlsx


In [39]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import os

def extract_shl_data(url):
    driver = webdriver.Edge()
    wait = WebDriverWait(driver, 10)
    driver.get(url)
    driver.maximize_window()

    # Accept cookies
    try:
        cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Allow all cookies')]")))
        cookie_btn.click()
    except:
        pass

    # Title
    try:
        title = driver.find_element(By.TAG_NAME, "h1").text.strip()
    except:
        title = "Not found"

    # Description
    try:
        desc_elem = driver.find_element(By.CSS_SELECTOR, ".product-catalogue-training-calendar__row.typ p")
        description = desc_elem.text.strip()
    except:
        description = "Not found"

    # Test Type
    try:
        test_type_spans = driver.find_elements(By.CSS_SELECTOR, "span.product-catalogue__key")
        test_type_values = [span.text.strip() for span in test_type_spans if span.text.strip()]
        test_type = ", ".join(test_type_values)
    except:
        test_type = "Not found"

    # Remote Testing
    try:
        driver.find_element(By.CSS_SELECTOR, "span.catalogue__circle.-yes")
        remote_testing = "Yes"
    except:
        remote_testing = "No"

    # Duration
    try:
        duration = "Not found"
        duration_elems = driver.find_elements(By.CSS_SELECTOR, ".product-catalogue-training-calendar__row p")
        for elem in duration_elems:
            text = elem.text.lower()
            if "minutes" in text:
                match = re.search(r'(\d+)\s*$', text)
                if match:
                    duration = match.group(1) + " minutes"
                    break
    except:
        duration = "Not found"

    driver.quit()

    return {
        "Assessment Name": title,
        "Description": description,
        "URL": url,
        "Remote Testing": remote_testing,
        "Duration": duration,
        "Test Type": test_type
    }

# List of URLs to scrape
urls = [
    "https://www.shl.com/solutions/products/product-catalog/view/adobe-experience-manager-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/adobe-photoshop-cc/",
    "https://www.shl.com/solutions/products/product-catalog/view/aeronautical-engineering-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/aerospace-engineering-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/agile-software-development/",
    "https://www.shl.com/solutions/products/product-catalog/view/agile-testing-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/ai-skills/",
    "https://www.shl.com/solutions/products/product-catalog/view/amazon-web-services-aws-development-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/android-development-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/angular-6-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/angularjs-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/apache-hadoop-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/global-skills-development-report/",
    "https://www.shl.com/solutions/products/product-catalog/view/net-framework-4-5/",
    "https://www.shl.com/solutions/products/product-catalog/view/net-mvc-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/net-mvvm-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/net-wcf-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/net-wpf-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/net-xaml-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/accounts-payable-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/accounts-payable-simulation-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/accounts-receivable-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/accounts-receivable-simulation-new/",
    "https://www.shl.com/solutions/products/product-catalog/view/ado-net-new/"
    
    
]

# Output Excel file
excel_file = "shl_tests.xlsx"

# Load existing data if file exists
if os.path.exists(excel_file):
    existing_df = pd.read_excel(excel_file)
else:
    existing_df = pd.DataFrame()

# Extract and append data
for url in urls:
    data = extract_shl_data(url)
    existing_df = pd.concat([existing_df, pd.DataFrame([data])], ignore_index=True)

# Save to Excel
existing_df.to_excel(excel_file, index=False)
print("Data saved to:", excel_file)


Data saved to: shl_tests.xlsx
